### hay conductores con dos colores
    - no son conductores, son empresas que dan servicio. Entonces vamos a hacer un tablon con todo.
      pero los datarmarts van a quitarselo de en medio
### cuantos viajes son en distinto dia?
    muy pocos viajes acaban en distinto dia -> dejar origen quitar destino, y puedes hacer una col de duracion del viaje

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder \
    .appName("SmallTestDataset") \
    .master("local[*]") \
    .getOrCreate()

In [4]:
#INPUT DATA
input_data_g = "/home/jupyter/proyect/ds_taxi_NY/green_tripdata_2024-01.parquet"
input_data_y = "/home/jupyter/proyect/ds_taxi_NY/yellow_tripdata_2024-01.parquet"

In [5]:
#FUNCIONES
def sumar_missing_per_var(x):
    return x.select([F.count( F.when(F.col(c).isNull(),c)  ).alias(c) for c in x.columns]).show()

In [6]:

df_y = spark.read.parquet(input_data_y)

In [7]:
#READ FILE GREEN
df_g = spark.read.parquet(input_data_g)

#SELECCION VARIABLES GREEN DATASET
lista_vars = ['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfg = df_g.select(lista_vars)

# TRANSFORMACION REDONDEO TIEMPO, Y DATE GREEN TAXI
dfg = dfg.withColumn("date_init_trip", F.date_format(F.col("lpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("lpep_pickup_datetime") ))\
         .withColumn("date_arrival_trip", F.date_format(F.col("lpep_dropoff_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_arrival_trip',     F.hour( F.col("lpep_dropoff_datetime") )) \
         .drop(F.col('lpep_pickup_datetime'))\
         .drop( F.col('lpep_dropoff_datetime') )

dfg = dfg.withColumn('tipoVehiculo', F.lit('Green'))

In [8]:
#READ YELLOW
df_y = spark.read.parquet(input_data_y)

#SELECCION VARIABLES YELLOW DATASET
lista_vars = ['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfy = df_y.select(lista_vars)


# TRANSFORMACION REDONDEO TIEMPO, Y DATE YELLOW TAXI
dfy = dfy.withColumn("date_init_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("tpep_dropoff_datetime") ))\
         .withColumn("date_arrival_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_arrival_trip',     F.hour( F.col("tpep_dropoff_datetime") )) \
         .drop(F.col('tpep_pickup_datetime'))\
         .drop( F.col('tpep_dropoff_datetime') )

dfy = dfy.withColumn('tipoVehiculo', F.lit('Yellow'))

In [9]:
dfy.columns == dfg.columns

True

In [10]:
# Union data sets
df = dfg.union(dfy)

In [11]:
# Orden columnas
df = df.select(['VendorID',
            'tipoVehiculo',
            'date_init_trip',
            'hour_init_trip',
            'date_arrival_trip',
            'hour_arrival_trip',              
            'PULocationID',
            'DOLocationID',
            'passenger_count',
            'trip_distance',
            'tip_amount',
            'total_amount'])

# Incluir propinas en el precio total
df = df.withColumn('total_amount', F.round(F.col('total_amount')+F.col('tip_amount'),2)  )

In [ ]:
# hay conductores con dos colores
df.groupBy(['VendorID']).agg(F.count('VendorID').alias('conteo') ).show()
df.groupBy(['VendorID']).agg(F.count('VendorID').alias('conteo') ).show()
df.select(['VendorID', 'tipoVehiculo']).distinct().groupBy(['VendorID']).agg(F.count('tipoVehiculo').alias('conteo') ).show()

+--------+-------+
|VendorID| conteo|
+--------+-------+
|       1| 737070|
|       2|2283845|
|       6|    260|
+--------+-------+

+--------+------+
|VendorID|conteo|
+--------+------+
|       1|     2|
|       6|     1|
|       2|     2|
+--------+------+



In [16]:
#viajes distinto dia (empieza y fin)

df.count()
df.filter(   F.col('date_init_trip')!=F.col('date_arrival_trip')).count()

430